In [2]:
#Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
path = r'C:\Users\alexa\OneDrive\Documents\data analytics\Achievement 4\01-09-2022 Instacart Basket Analysis'

In [7]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'order_busy_frequencies.pkl'))
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [8]:
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [11]:
#4 question: rename columns
df_cust.rename(columns = {'Surnam': 'Last Name'}, inplace=True)
df_cust.rename(columns = {'STATE': 'State'}, inplace=True)
df_cust.rename(columns = {'date_joined': 'Date joined'}, inplace=True)
df_cust.rename(columns = {'n_dependants': 'No of dependents'}, inplace=True)
df_cust.rename(columns = {'fam_status': 'Marital status'}, inplace=True)

In [12]:
df_cust.head()

,user_id,First Name,Last Name,Gender,State,Age,Date joined,No of dependents,Marital status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [13]:
df_cust.shape

(206209, 10)

In [14]:
#5 question: consistency check (duplicates, missing values, mixed data types)
#A check for duplicates:
df_cust[df_cust.duplicated()]

,user_id,First Name,Last Name,Gender,State,Age,Date joined,No of dependents,Marital status,income


In [15]:
#B check for missing values
df_cust.isnull().sum()

user_id                 0
First Name          11259
Last Name               0
Gender                  0
State                   0
Age                     0
Date joined             0
No of dependents        0
Marital status          0
income                  0
dtype: int64

In [18]:
df_nan = df_cust[df_cust['First Name'].isnull() == True]

In [19]:
df_nan

,user_id,First Name,Last Name,Gender,State,Age,Date joined,No of dependents,Marital status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


that makes 5.45% of the data in the 'First Name' column is missing

In [20]:
#C checking for mix data types
df_cust.dtypes

user_id              int64
First Name          object
Last Name           object
Gender              object
State               object
Age                  int64
Date joined         object
No of dependents     int64
Marital status      object
income               int64
dtype: object

In [21]:
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

First Name


the 'First Name' column, on top of having 5.45% missing data it also has mixed data

In [22]:
df_cust['First Name'] = df_cust['First Name'].astype('str')

In [23]:
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

by correcting the data type in the "First Name' column, it is now no longer a mixed data type column

In [24]:
#6 question: combining the customer dataframe to the order_busy_frequencies.pkl
df_ords_prods_merge.head()

,order_id,user_id,order_number,Order_day_of_week,order_hour_of_day,Days since last order,New customer,product_id,add_to_cart_order,reordered,...,price_range_loc,Busiest day,2 busy/slow days,busiest_period_of_day,max order,loyalty_flag,Mean_price,Spend_flag,Median_since_last_ordered,Order_frequency_flag
0,2539329,1,1,2,8,NaN,True,196.0,1.0,0.0,...,Mid range products,Regular busy,Regular busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent cust
1,2398795,1,2,3,7,15.0,False,196.0,1.0,1.0,...,Mid range products,Regular busy,Two slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent cust
2,473747,1,3,3,12,21.0,False,196.0,1.0,1.0,...,Mid range products,Regular busy,Two slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent cust
3,2254736,1,4,4,7,29.0,False,196.0,1.0,1.0,...,Mid range products,Least busy,Two slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent cust
4,431534,1,5,4,15,28.0,False,196.0,1.0,1.0,...,Mid range products,Least busy,Two slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent cust


In [25]:
#note: the key column is the user_id
df_ords_prods_merge.dtypes

order_id                        int64
user_id                         int64
order_number                    int64
Order_day_of_week               int64
order_hour_of_day               int64
Days since last order         float64
New customer                     bool
product_id                    float64
add_to_cart_order             float64
reordered                     float64
product_name                   object
aisle_id                        int64
department_id                   int64
prices                        float64
_merge                       category
price_range_loc                object
Busiest day                    object
2 busy/slow days               object
busiest_period_of_day          object
max order                       int64
loyalty_flag                   object
Mean_price                    float64
Spend_flag                     object
Median_since_last_ordered     float64
Order_frequency_flag           object
dtype: object

both user_id columns are int64 types

In [27]:
df_merged = df_ords_prods_merge.merge(df_cust, on = 'user_id')

In [28]:
#export merged dataframe
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_cust_combined.pkl'))